<a href="https://colab.research.google.com/github/Andrian0s/ML4NLP1-2023-Tutorial-Notebooks/blob/main/tutorial_notebooks/03_tutorial_skorch_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML4NLP1
## Tutorial 03: sklearn: Pipeline and skorch

# Installing skorch and loading libraries

In [ ]:
import subprocess

# Installation on Google Colab
try:
    import google.colab
    subprocess.run(['python', '-m', 'pip', 'install', 'skorch'])
except ImportError:
    pass

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
from skorch import NeuralNetClassifier

In [ ]:
torch.manual_seed(0)
torch.cuda.manual_seed(0)

In [ ]:
import pandas as pd
import numpy as np
import csv
import re
import string
from collections import defaultdict

## Training a classifier and making predictions

In [ ]:
# download dataset
!gdown 1IUyw6n3IrabkhPQH7Jkuw_Vml4UXwiCI # y_train
!gdown 1IVeWdIaO5tXPeMOOsHPjqd1K3Obp4tC3 # y_test
!gdown 1ITAYFokSjAxaIHh6bzv5id-phSqx78q9 # X_train
!gdown 1IZrB2mCAmkly0w5pdWhUlMqJdxBC8OtK # X_test

In [ ]:
X_train = np.load('X_train.npy', allow_pickle=True)
X_test = np.load('X_test.npy', allow_pickle=True)
y_train = np.load('y_train.npy')
y_test = np.load('y_test.npy')


In [ ]:
X_train.shape

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC

In [ ]:
# Preprocessing
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)  # num_docs x num_words
X_test_counts = count_vect.transform(X_test)
tfidf_tranformer = TfidfTransformer(smooth_idf=True).fit(X_train_counts)
X_train_tfidf = tfidf_tranformer.transform(X_train_counts)
X_test_tfidf = tfidf_tranformer.transform(X_test_counts)
label_encoder = LabelEncoder()
le_fitted = label_encoder.fit(y_train)

In [ ]:
le_fitted.classes_

In [ ]:
print(len(le_fitted.classes_))

In [ ]:
y_train = le_fitted.transform(y_train)
y_test = le_fitted.transform(y_test)

In [ ]:
# What shape is this
X_train_tfidf.shape

In [ ]:
X = X_train_tfidf.astype(np.float32)
y = y_train.astype(np.int64)

In the following, we define a vanilla neural network with two hidden layers. The output layer should have as many outputs as there are classes. In addition, it should have a nonlinearity function.

In [ ]:
class ClassifierModule(nn.Module):
    def __init__(
            self,
            num_units=7500,
            nonlin=F.relu,
    ):
        super(ClassifierModule, self).__init__()
        self.num_units = num_units
        self.nonlin = nonlin

        self.dense0 = nn.Linear(130107, num_units)
        self.nonlin = nonlin
        self.dense1 = nn.Linear(num_units, 200)
        self.output = nn.Linear(200, 20)

    def forward(self, X, **kwargs):
      X = self.nonlin(self.dense0(X))
      X = F.relu(self.dense1(X))
      X = self.output(X)
      return X.squeeze(dim=1)

In [ ]:
net = NeuralNetClassifier(
    ClassifierModule,
    max_epochs=10,
    criterion=nn.CrossEntropyLoss(),
    lr=0.1,
    device='cuda',  # comment this to train with CPU
)

In [ ]:
net.fit(X, y)